<h1 style="text-align: center;" markdown="1">Machine Learning Algorithms for Poverty Prediction</h1> 
<h2 style="text-align: center;" markdown="2">A project of the World Bank's Knowledge for Change Program</h2>
<h3 style="text-align: center;" markdown="3">(KCP, Grant TF0A4534)</h3>


> *This notebook is part of a series that has been developed as an empirical comparative assessment of machine learning classification algorithms applied to poverty prediction. The objectives of this project are to explore how well machine learning algorithms perform when given the task to identify the poor in a given population, and to provide a resource of machine learning techniques for researchers, data scientists, and statisticians in developing countries.*

## Process Additional Years of Indonesia Surveys <a class="anchor" id="description"></a>

In this notebook, we prepare additional data sets for testing algorithms towards the end of the project. The preparation process is exactly the same, except that this time we don't need any training data (the algortihms are already trained)

This data prepares Indonesia data from three new years,

* 2011
* 2013
* 2014

As mentioned above, the process is the same. The only exception is that new data is compared against the original 2012 data such that the columns (questions) of all data sets are equal. For a few columns, this required mapping different names to the original data set, which we point out below.

### Raw data overview
The data for this project includes data and metadata from surveys in Indonesia for the years 2011,m 2013, 2014. For each year, you will find:
* The survey questionnaire(s)
* Two Stata (version 14) data files: household and individual. These data contain a subset of the full survey dataset
* An Excel file, providing a list of variables in the data files
* Two text files (output of the “codebook” Stata command, providing a list of variables with some metadata.
 
### Stata file contents
The Stata data files contain:
* The label variable (the binary variable to be predicted), named “poor”. This is just a poor/non-poor dummy variable. Note: poverty is calculated at the household level; either all or none of one same household member is/are poor.
* The sample weights (the objective is to “predict” poverty taking into account these sample weights. Variable `wta_hh` contains the household weight; variable `wta_pop` contains the population weight (household weight * household size). These are probability weights (the sum of `wta_pop` gives us the extrapolated population of the country).
* The file that will be used as training/test set is the `household.dta` file.
* The `individual.dta` file contains variables on each household member. The file is provided to allow us to create additional variables to be added to the `household.dta` file. The variable `hhsize` (household size) is one such derived variable, already calculated. We will probably use this to generate new variables to possibly improve the accuracy of the model.

### Filtering Consumable Feaures
The `household.dta` file contains a series of variables whose name starts with `cons_`. These are variables derived from the expenditure module of the survey. It indicates whether a household consumed/purchased a specific item or not. Any reported value > 0 for an item was transformed into code 1; 0 otherwise.  

*IMPORTANT*: Since one objective of this project is to design light survey questionnaires that would allow accurate poverty prediction, we want to be selective in the list of variables we use in the final model. What we would like to do is optimize a model using the variables we find useful (including new derived variables), under the constraint that no more than 50 of the `cons_` variables will be used. Some quick analysis of these `cons_` variables should make it possible to make a selection prior to tuning the models.

## Load the Raw Data <a class="anchor" id="load-data"></a>

First, we load a few essential modules used in notebook. We have developed several utility functions in the `load_data.py` file located in the `src/data` directory that will be used throughout this project for convenience.

In [1]:
%matplotlib inline

import os
import sys
import json

import numpy as np
import pandas as pd
from pandas.io.stata import StataReader

from matplotlib import pyplot as plt
from IPython.display import display
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split

# Add our local functions to the path
sys.path.append(os.path.join(os.pardir, 'src'))
from data import load_data
from features import process_features

## Load with same function as before <a class="anchor" id="load"></a>

In [2]:
def load_stata_file(filepath, 
                    index_cols, 
                    update_education=False, 
                    drop_minornans=False, 
                    drop_unlabeled=False):
    """ Load data and metadata from Stata file"""
    data = pd.read_stata(filepath, convert_categoricals=False).set_index(index_cols)

    with StataReader(filepath) as reader:
        reader.value_labels()
            
        mapping = {col: reader.value_label_dict[t] for col, t in 
                   zip(reader.varlist, reader.lbllist)
                   if t in reader.value_label_dict}
        
        # manually update some specific columns
        # in the Malawi dataset
        if update_education and 'ind_educ09' in mapping:
            mapping['ind_educ09'][13] = "Primary - " + mapping['ind_educ09'][13]
            mapping['ind_educ09'][14] = "Primary - " + mapping['ind_educ09'][14]
            
            mapping['ind_educ09'][23] = "Secondary - " + mapping['ind_educ09'][23]
            mapping['ind_educ09'][24] = "Secondary - " + mapping['ind_educ09'][24]
        
        data.replace(mapping, inplace=True)
        
        # convert the categorical variables into
        # the category type
        for c in data.columns:
            if c in mapping:
                data[c] = data[c].astype('category')
                        
        # drop records with only a few nans
        if drop_minornans: 
            nan_counts = (data.applymap(pd.isnull)
                          .sum(axis=0)
                          .sort_values(ascending=False))
            nan_cols = nan_counts[(nan_counts > 0) & (nan_counts < 10)].index.values
            data = data.dropna(subset=nan_cols)
        # drop unlabeled categorical values
        def find_unlabeled(x):
            if x.name in mapping.keys():
                return [val if (val in mapping[x.name].values() or pd.isnull(val)) 
                        else 'UNLABELED' for val in x]
            else:
                return x
            
        data = data.apply(find_unlabeled)
        data = data[~data.applymap(lambda x: x == "UNLABELED").any(axis=1)]
        
        # read the actual questions that were asked for reference
        questions = reader.variable_labels()
        
    return data, questions

## Here is a simple function to help us find mismatched columns

In [3]:
from difflib import get_close_matches

def find_mismatched_columns(new, old):
    in_new_but_not_old = set(new.columns.values) - set(old.columns.values)

    # don't want to kill the weights and labels, they are loaded sep so won't appear here
    for col in ['wta_pop', 'wta_hh', 'poor']:
        in_new_but_not_old.discard(col)

    in_old_but_not_new = set(old.columns.values) - set(new.columns.values)
    return in_new_but_not_old, in_old_but_not_new

And here is a map between column names to that all features are the same for 2011-2014

In [4]:
# manually created column map               
col_name_map = {
'hld_bwwater__Pump': 'hld_bwwater__Pipe with meter',
'hld_bwwater__Retail piping water': 'hld_bwwater__Pipe, retail payment',
'hld_bwwater__Others': 'hld_bwwater__Other',
'hld_dwater__Recycled bottled/ refill water': 'hld_dwater__Recycled bottled water',
'hld_wall__Concrete/brick': 'hld_wall__Concrete',
'hld_bwwater__Protected well': 'hld_bwwater__Protected/covered well',
'hld_toilet__Pit latrine': 'hld_toilet__Pit toilet/plengsengan',
'hld_dwater__Meter piping meter': 'hld_dwater__Pipe with meter',
'hld_bwwater__Unprotected well': 'hld_bwwater__Unprotected/uncovered well',
'hld_bwwater__Rain water': 'hld_bwwater__Rainwater',
}

### We now load and process the new years: 2011, 2013, 2014

Along the way, we'll keep track of mismatched columns (printed in the cell output)

In [5]:
# for tracking mismatches below
mismatches = {}

for year in [2011, 2013, 2014]:
    
    print(f'\t\t---- {year} ----')

    # Load Indonesia household data
    filepath = eval(f'load_data.IDN_{year}_HOUSEHOLD')
    idn_hhold, idn_hhold_questions = load_stata_file(filepath, ['hid'])
    s = 'Indonesia household data has {:,} rows and {:,} columns'
    print(s.format(idn_hhold.shape[0], idn_hhold.shape[1]))
    s = 'Percent poor: {:0.1%} \tPercent non-poor: {:0.1%}'
    print(s.format(idn_hhold.poor.value_counts(normalize=True).loc['Poor'], 
                   idn_hhold.poor.value_counts(normalize=True).loc['Non-poor']))
    idn_hhold.head()

    # Load Indonesia individual data
    filepath = eval(f'load_data.IDN_{year}_INDIVIDUAL')
    idn_indiv, idn_indiv_questions = load_stata_file(filepath, ['hid', 'iid'])
    s = 'Indonesia individual data has {:,} rows and {:,} columns'
    print(s.format(idn_indiv.shape[0], idn_indiv.shape[1]))
    s = 'Percent poor: {:0.1%} \tPercent non-poor: {:0.1%}'
    print(s.format(idn_indiv.poor.value_counts(normalize=True).loc['Poor'], 
                   idn_indiv.poor.value_counts(normalize=True).loc['Non-poor']))

    idn_hhold.poor = (idn_hhold.poor == 'Poor')

    def add_derived_feature(df, 
                            feature_name, 
                            values, 
                            questions_dict, 
                            question=None):
        '''Add a derived feature to the household dataframe and update questions'''
        df['der_' + feature_name] = values
        questions_dict['der_' + feature_name] = question
        return

    # IDN - Number of children (10 and under) in household
    values = (idn_indiv.ind_age <= 10).sum(level=0).astype(int)
    question = 'Number of children in household age 10 and under'
    add_derived_feature(idn_hhold, 
                        'nchild10under', 
                        values, 
                        idn_hhold_questions, 
                        question=question)

    # IDN - Number of males and females in household over age 10
    values = ((idn_indiv.ind_sex == 'Male') & (idn_indiv.ind_age > 10)).sum(level=0).astype(int)
    question = 'Number of males in household over age 10'
    add_derived_feature(idn_hhold, 
                        'nmalesover10', 
                        values, 
                        idn_hhold_questions, 
                        question=question)
    values = ((idn_indiv.ind_sex == 'Female') & (idn_indiv.ind_age > 10)).sum(level=0).astype(int)
    question = 'Number of females in household over age 10'
    add_derived_feature(idn_hhold, 
                        'nfemalesover10', 
                        values, 
                        idn_hhold_questions, 
                        question=question)

    # IDN - Number of household members who can read and write in any language
    values = (idn_indiv.ind_educ11 == 'Yes').sum(level=0).astype(int)
    question = 'Number of household members who can read and write'
    add_derived_feature(idn_hhold, 
                        'nliterate', 
                        values, 
                        idn_hhold_questions, 
                        question=question)

    # IDN - Number of household members employed in past 3 months
    values = (idn_indiv.ind_work3 == 'Yes').sum(level=0).astype(int)
    question = 'Number of household members employed in past 3 months (over age 10)'
    add_derived_feature(idn_hhold, 
                        'nemployedpast3mo',
                        values,
                        idn_hhold_questions,
                        question=question)

    # IDN - Number of household members who accessed the internet in past 3 months
    values = (idn_indiv.ind_educ15 == 'Yes').sum(level=0).astype(int)
    question = 'Number of household members who accessed the internet in past 3 months'
    add_derived_feature(idn_hhold, 
                        'ninternetpast3mo',
                        values,
                        idn_hhold_questions,
                        question=question)

    # create dummy variables for categoricals
    idn_hhold = pd.get_dummies(idn_hhold, drop_first=True, dummy_na=True, prefix_sep='__')

    print("Indonesia household with dummy variables added", idn_hhold.shape)

    # remove columns with only one unique value (all nan dummies from columns with no missing values)
    idn_hhold = idn_hhold.loc[:, idn_hhold.nunique(axis=0) > 1]

    print("Indonesia household with constant columns dropped", idn_hhold.shape)

    # remove duplicate columns - these end up being all from nan or Not Applicable dummies 
    process_features.drop_duplicate_columns(idn_hhold, ignore=['wta_hh', 'wta_pop'], inplace=True)

    print("Indonesia household shape with duplicate columns dropped", idn_hhold.shape)

    # it's all test data, just need test path
    _, TEST_PATH, QUESTIONS_PATH = load_data.get_country_filepaths(f'idn-{year}')
    
    # select out columns not in trained model, zero-fill columns not in new data, try to match names where possible
    _, TEST_PATH_TO_MATCH, QUESTIONS_PATH_TO_MATCH = load_data.get_country_filepaths('idn')
    test_to_match, _, _ = load_data.load_data(TEST_PATH_TO_MATCH)
    
    print(f'Reducing to {test_to_match.columns.values.shape[0]} features...\n')
    
    idn_hhold.rename(col_name_map, axis=1, inplace=True)   
    
    in_new_but_not_old, in_old_but_not_new = find_mismatched_columns(idn_hhold, test_to_match)
    
    close_matches = {new_col: get_close_matches(new_col, in_old_but_not_new) for new_col in in_new_but_not_old}
    mismatches[year] = close_matches

    # drop any remaining mismatches
    in_new_but_not_old, in_old_but_not_new = find_mismatched_columns(idn_hhold, test_to_match)
    
    print(f'\nAfter rename, in new data but not old:\t{in_new_but_not_old}')
    print('dropping...')
    idn_hhold.drop(in_new_but_not_old, inplace=True, axis=1)
    
    print(f'In old data but not new:\t{in_old_but_not_new}')
    print('filling...\n\n')
    
    for col in in_old_but_not_new:
        idn_hhold[col] = 0
        idn_hhold[col] = 0
    

    idn_hhold.to_pickle(TEST_PATH)
    with open(QUESTIONS_PATH, 'w') as fp:
        json.dump(idn_hhold_questions, fp)

		---- 2011 ----
Indonesia household data has 71,932 rows and 359 columns
Percent poor: 11.1% 	Percent non-poor: 88.9%
Indonesia individual data has 284,539 rows and 53 columns
Percent poor: 13.8% 	Percent non-poor: 86.2%
Indonesia household with dummy variables added (71932, 814)
Indonesia household with constant columns dropped (71932, 463)
Indonesia household shape with duplicate columns dropped (71932, 455)
Reducing to 453 features...


After rename, in new data but not old:	{'hld_cooking__Never cooking', 'hld_toilet__Water seal latrine'}
dropping...
In old data but not new:	{'hld_toilet__Squat toilet/cemplung', 'hld_bwwater__Recycled bottled water', 'hld_bwwater__Terrestrial well/pump'}
filling...


		---- 2013 ----
Indonesia household data has 70,842 rows and 356 columns
Percent poor: 7.7% 	Percent non-poor: 92.3%
Indonesia individual data has 274,051 rows and 55 columns
Percent poor: 9.8% 	Percent non-poor: 90.2%
Indonesia household with dummy variables added (70842, 811)
Indone

## Display the mismatches for each year

In [6]:
for year, mismatch in mismatches.items():
    print('\n**************************\n')
    print(f'{year}')
    print('**************************\n')
    for new_col, matches in mismatch.items():
        print(f'{new_col}:')
        if not matches:
            print(f'\t\tNO MATCHES FOUND (with >= .6 probability)')
        for match in matches:
            print(f'\t\t{match}')


**************************

2011
**************************

hld_cooking__Never cooking:
		NO MATCHES FOUND (with >= .6 probability)
hld_toilet__Water seal latrine:
		NO MATCHES FOUND (with >= .6 probability)

**************************

2013
**************************

hld_cooking__Never cooking:
		NO MATCHES FOUND (with >= .6 probability)
hld_cope1__Not applicable:
		NO MATCHES FOUND (with >= .6 probability)
hld_toilet__Water seal latrine:
		NO MATCHES FOUND (with >= .6 probability)

**************************

2014
**************************

hld_cooking__Never cooking:
		NO MATCHES FOUND (with >= .6 probability)
hld_toilet__Water seal latrine:
		NO MATCHES FOUND (with >= .6 probability)


The data are now ready for prediction!